In [0]:
!kill -9 -1

# **Access Colaboratory**

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

In [0]:
from google.colab import auth
auth.authenticate_user()

from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [4]:
!ls
!ls drive

datalab  drive
Colab Notebooks  Colaboratory  iMaterialist  Project Design.odt


# **Libraries**

In [5]:
import h5py, json, time, os
import cv2, numpy as np

from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD

from matplotlib import pyplot as plt

from keras.utils import np_utils

%matplotlib inline

from keras import backend as K

K.set_image_dim_ordering('th')

Using TensorFlow backend.


## Load Classes

In [0]:
os.chdir('/content/drive')

In [7]:
test_path="iMaterialist/validation_dataset/"
test_dataset_name = 'validation_last'
with h5py.File(test_path+'{}_labels.h5'.format(test_dataset_name), 'r') as hf:
    test_set_y_orig = np.array(hf['{}_labels'.format(test_dataset_name)][:])

classes = []
for i in range (1,len(test_set_y_orig)):
    if test_set_y_orig[i] not in classes:
        classes.append(test_set_y_orig[i])

classes = np.array(classes) # the list of classes

print(classes.shape)

test_set_y_orig = None

(128,)


## Time

In [0]:
def elapsed (start):
    """
    Returns elapsed time in hh:mm:ss format from start time in unix format
    """
    elapsed = time.time()-start
    hours, rem = divmod(elapsed, 3600)
    minutes, seconds = divmod(rem, 60)
    return("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

# **Load DataSets**

In [0]:
def load_trainDS(train_path, train_dataset_name, batch_size=1000):
    #train_path="iMaterialist/train_dataset/"
    #train_dataset_name = 'train_1'

    # Train dataset
    with h5py.File(train_path+'{}_images.h5'.format(train_dataset_name), 'r') as hf:
        train_set_x_orig = np.array(hf['{}_images'.format(train_dataset_name)][batch_size-1000:batch_size])   
    with h5py.File(train_path+'{}_labels.h5'.format(train_dataset_name), 'r') as hf:
        train_set_y_orig = np.array(hf['{}_labels'.format(train_dataset_name)][batch_size-1000:batch_size])

    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig

In [0]:
def load_testDS(test_path, test_dataset_name, batch_size=1000):
    #test_path="iMaterialist/validation_dataset/"
    #test_dataset_name = 'validation_last'

    # Test dataset (validation)
    with h5py.File(test_path+'{}_images.h5'.format(test_dataset_name), 'r') as hf:
        test_set_x_orig = np.array(hf['{}_images'.format(test_dataset_name)][batch_size-1000:batch_size])   
    with h5py.File(test_path+'{}_labels.h5'.format(test_dataset_name), 'r') as hf:
        test_set_y_orig = np.array(hf['{}_labels'.format(test_dataset_name)][batch_size-1000:batch_size])

    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return test_set_x_orig, test_set_y_orig

In [11]:
!free -m

              total        used        free      shared  buff/cache   available
Mem:          13029        1454         204         223       11369       11103
Swap:             0           0           0


In [0]:
train_path="iMaterialist/train_dataset/"
test_path="iMaterialist/validation_dataset/"
train_dataset_name = 'train_1'
test_dataset_name = 'validation_last'

x_train, y_train = load_trainDS(train_path, train_dataset_name)
x_test, y_test  = load_testDS(test_path, test_dataset_name)

In [16]:
print(x_train.shape)
print(y_test.shape)

(1000, 300, 300, 3)
(1000,)


In [0]:
x_train = x_train.reshape(x_train.shape[0], 3, 300, 300).astype('float32') #(1000, 1, 28, 28)
x_test = x_test.reshape(x_test.shape[0], 3, 300, 300).astype('float32') #(1000, 1, 28, 28)

In [0]:
x_train = x_train/255
x_test = x_test/255

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

num_classes = y_test.shape[1]

In [23]:
print(x_train.shape)
print(y_test.shape)

(1000, 3, 300, 300)
(1000, 129, 2)


In [24]:
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(3,300,300)))
model.add(Convolution2D(64, (3, 3), activation='relu', name='conv1_1'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu', name='conv1_2'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu', name='conv2_1'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu', name='conv2_2'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu', name='conv3_1'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu', name='conv3_2'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu', name='conv3_3'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu', name='conv4_1'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu', name='conv4_2'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu', name='conv4_3'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu', name='conv5_1'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu', name='conv5_2'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu', name='conv5_3'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1000, activation='softmax', name='predict'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_14 (ZeroPaddi (None, 3, 302, 302)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 64, 300, 300)      1792      
_________________________________________________________________
zero_padding2d_15 (ZeroPaddi (None, 64, 302, 302)      0         
_________________________________________________________________
conv1_2 (Conv2D)             (None, 64, 300, 300)      36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 64, 150, 150)      0         
_________________________________________________________________
zero_padding2d_16 (ZeroPaddi (None, 64, 152, 152)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 128, 150, 150)     73856     
__________

In [20]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=200)

ValueError: ignored

In [0]:
model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'
del model  # deletes the existing model

# returns a compiled model
# identical to the previous one
model = load_model('my_model.h5')

In [0]:
scores = model.evaluate(x_test, y_test, verbose=0)
print("\nacc: %.2f%%" % (scores[1]*100))